In [2]:
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch import nn
from torch.nn import functional as F
from torch import optim

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification

In [4]:
X,Y = make_classification(n_samples=1000,n_features=16,n_classes=2)

In [7]:
Y= Y.reshape(-1,1)

In [36]:
X.shape,Y.shape,X[:10],Y[:10]

((1000, 16),
 (1000, 1),
 array([[ 0.09746659, -0.92047125,  0.24777324,  0.67787059, -0.13692098,
         -1.39142208,  0.29069977,  0.31368041,  0.62104336,  0.56792336,
          0.22786059, -0.30298162,  0.65325609, -0.33254188, -2.34684868,
          0.10869357],
        [ 0.73140171, -0.01458245,  0.13410256, -1.06409572, -1.52624856,
         -0.48079001, -0.75185376, -0.20263266,  1.15502683, -0.78376276,
         -0.30530528,  0.62627747, -1.39539359, -1.27262034, -2.5910644 ,
         -0.88052991],
        [-0.27929825,  0.33571878, -1.39250085, -0.02264971, -0.27276476,
         -0.20796883,  0.26922866, -0.49917626, -0.28436934, -0.65474016,
         -1.12181408, -1.25327386, -0.45811138,  0.77988883,  0.77975413,
          0.31106297],
        [-0.69332156, -0.04587563,  0.60131044, -0.23728221, -0.12263886,
         -0.83272037,  1.43087416,  1.27831888,  0.32366909,  1.50846544,
         -0.7843675 ,  0.33598538,  0.780358  ,  0.5960006 , -0.26952243,
         -1.123577

# 机器学习方法

In [62]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

In [63]:
reg = SVC()
reg.fit(X,Y.ravel())

SVC()

In [64]:
accuracy_score(reg.predict(X),Y)

0.922

# 用来做随机batch的模块

In [65]:
class DiabetesDataset(Dataset):
    def __init__(self, x_, y_):
        self.len = x_.shape[0]
        self.x_data = torch.from_numpy(x_)
        # 这部很关键， 特征类型要求float类型
        self.x_data = self.x_data.float()

        self.y_data = torch.from_numpy(y_)
        self.y_data = self.y_data.float()


    def __getitem__(self, item):
        return self.x_data[item], self.y_data[item]

    def __len__(self):
        return self.len

In [66]:
dataset = DiabetesDataset(X, Y)
train_loader = DataLoader(dataset=dataset, batch_size=100, shuffle=True)

# 网络结构

In [67]:
#网络结构
class NeuralNet(nn.Module):
    def __init__(self, input_size_, hidden_size_, num_classes_):
        super().__init__()
        self.fc1 = nn.Linear(input_size_, hidden_size_) 
        self.fc2 = nn.Linear(hidden_size_, num_classes_)  
    
    def forward(self, x):
        out = self.fc1(x)
        out = F.relu(out)
        out = self.fc2(out)
        #二分类最后一步进行sigmoid
        out = torch.sigmoid(out)
        return out

# 建立模型训练

In [68]:
model = NeuralNet(16, 8, 1)
#损失函数
criterion = nn.BCELoss(reduction='mean')
#优化算法
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [69]:
total_step = len(train_loader)
num_epochs = 1000
for epoch in range(num_epochs):
    for i, (x, y) in enumerate(train_loader):
        # Reshape images to (batch_size, input_size)
        x = x.reshape(-1, 16)
        # Forward pass
        outputs = model(x)
        loss = criterion(outputs, y)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        #输出中间信息
        if i % 50 == 0 and epoch%20==0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
                  .format(epoch + 1, num_epochs, i + 1, total_step, loss.item()))

Epoch [1/1000], Step [1/10], Loss: 0.7150
Epoch [21/1000], Step [1/10], Loss: 0.1926
Epoch [41/1000], Step [1/10], Loss: 0.1919
Epoch [61/1000], Step [1/10], Loss: 0.1778
Epoch [81/1000], Step [1/10], Loss: 0.1212
Epoch [101/1000], Step [1/10], Loss: 0.2404
Epoch [121/1000], Step [1/10], Loss: 0.1861
Epoch [141/1000], Step [1/10], Loss: 0.1254
Epoch [161/1000], Step [1/10], Loss: 0.1421
Epoch [181/1000], Step [1/10], Loss: 0.1265
Epoch [201/1000], Step [1/10], Loss: 0.1603
Epoch [221/1000], Step [1/10], Loss: 0.0616
Epoch [241/1000], Step [1/10], Loss: 0.0944
Epoch [261/1000], Step [1/10], Loss: 0.0835
Epoch [281/1000], Step [1/10], Loss: 0.1321
Epoch [301/1000], Step [1/10], Loss: 0.1165
Epoch [321/1000], Step [1/10], Loss: 0.0792
Epoch [341/1000], Step [1/10], Loss: 0.1228
Epoch [361/1000], Step [1/10], Loss: 0.0760
Epoch [381/1000], Step [1/10], Loss: 0.0845
Epoch [401/1000], Step [1/10], Loss: 0.0700
Epoch [421/1000], Step [1/10], Loss: 0.0766
Epoch [441/1000], Step [1/10], Loss: 0

# 预测

In [70]:
predicted = model(torch.from_numpy(X).float())

In [71]:
# 预测出来的结果要转换01分布
predicted_01 = torch.where(predicted>0.5,torch.tensor(1),torch.tensor(0))

In [72]:
accuracy_score(predicted_01.detach().numpy(),Y)

0.962